In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", 130)
pd.set_option("display.width", 140)

#Se lee dataset con solo los casos de cancer de origen desconocido en diagnostico primario

df = pd.read_csv("../data/raw/df_cup_primary.csv")

df.head()

,COD_HOSPITAL,ID_BENEFICIARIO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,TIPO_PROCEDENCIA,TIPO_INGRESO,ESPECIALIDAD_MEDICA,TIPO_ACTIVIDAD,FECHA_INGRESO,SERVICIOINGRESO,FECHATRASLADO1,SERVICIOTRASLADO1,FECHATRASLADO2,SERVICIOTRASLADO2,FECHATRASLADO3,SERVICIOTRASLADO3,FECHATRASLADO4,SERVICIOTRASLADO4,FECHATRASLADO5,SERVICIOTRASLADO5,FECHATRASLADO6,SERVICIOTRASLADO6,FECHATRASLADO7,SERVICIOTRASLADO7,FECHATRASLADO8,SERVICIOTRASLADO8,FECHATRASLADO9,SERVICIOTRASLADO9,FECHAALTA,SERVICIOALTA,TIPOALTA,CONDICIONDEALTANEONATO1,PESORN1,SEXORN1,RN1ESTADO,CONDICIONDEALTANEONATO2,PESORN2,SEXORN2,RN2ESTADO,CONDICIONDEALTANEONATO3,PESORN3,SEXORN3,RN3ESTADO,CONDICIONDEALTANEONATO4,PESORN4,SEXORN4,RN4ESTADO,DIAGNOSTICO1,DIAGNOSTICO2,DIAGNOSTICO3,DIAGNOSTICO4,DIAGNOSTICO5,DIAGNOSTICO6,DIAGNOSTICO7,DIAGNOSTICO8,DIAGNOSTICO9,DIAGNOSTICO10,DIAGNOSTICO11,DIAGNOSTICO12,DIAGNOSTICO13,DIAGNOSTICO14,DIAGNOSTICO15,DIAGNOSTICO16,DIAGNOSTICO17,DIAGNOSTICO18,DIAGNOSTICO19,DIAGNOSTICO20,DIAGNOSTICO21,DIAGNOSTICO22,DIAGNOSTICO23,DIAGNOSTICO24,DIAGNOSTICO25,DIAGNOSTICO26,DIAGNOSTICO27,DIAGNOSTICO28,DIAGNOSTICO29,DIAGNOSTICO30,DIAGNOSTICO31,DIAGNOSTICO32,DIAGNOSTICO33,DIAGNOSTICO34,DIAGNOSTICO35,PROCEDIMIENTO1,PROCEDIMIENTO2,PROCEDIMIENTO3,PROCEDIMIENTO4,PROCEDIMIENTO5,PROCEDIMIENTO6,PROCEDIMIENTO7,PROCEDIMIENTO8,PROCEDIMIENTO9,PROCEDIMIENTO10,PROCEDIMIENTO11,PROCEDIMIENTO12,PROCEDIMIENTO13,PROCEDIMIENTO14,PROCEDIMIENTO15,PROCEDIMIENTO16,PROCEDIMIENTO17,PROCEDIMIENTO18,PROCEDIMIENTO19,PROCEDIMIENTO20,PROCEDIMIENTO21,PROCEDIMIENTO22,PROCEDIMIENTO23,PROCEDIMIENTO24,PROCEDIMIENTO25,PROCEDIMIENTO26,PROCEDIMIENTO27,PROCEDIMIENTO28,PROCEDIMIENTO29,PROCEDIMIENTO30,MEDICOINTERV1_ENCRIPTADO,FECHAPROCEDIMIENTO1,FECHAINTERV1,ESPECIALIDADINTERVENCION,MEDICOALTA_ENCRIPTADO,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA
0,114101,67717452,MUJER,1952-11-11,NINGUNO,CORDILLERA,PUENTE ALTO,CHILE,FONASA INSTITUCIONAL - (MAI) B,METROPOLITANO SURORIENTE,SERVICIO EMERGENCIA (DOMICILIO),URGENCIA,MEDICINA INTERNA,HOSPITALIZACIÓN,2024-09-20,DERIVACIÓN MÉDICO QUIRÚRGICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-09-23,DERIVACIÓN MÉDICO QUIRÚRGICO,DOMICILIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C78.7,C54.1,C78.6,R17,K80.0,I87.1,M16.9,I10,Z92.2,Z90.4,Z90.7,Z90.8,Z87.1,Z92.6,Z96.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.01,88.76,89.52,99.29,99.21,99.19,99.26,89.66,90.59,89.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84372283,NaN,74113,"1,3247",3,3,NaN
1,114101,67782077,MUJER,1948-02-02,NINGUNO,SANTIAGO,LA FLORIDA,CHILE,FONASA INSTITUCIONAL - (MAI) B,METROPOLITANO SURORIENTE,SERVICIO EMERGENCIA (DOMICILIO),URGENCIA,ONCOLOGÍA MÉDICA,HOSPITALIZACIÓN,2024-11-12,GINECOLOGÍA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-11-13,GINECOLOGÍA,DOMICILIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C78.6,C56,C78.5,C78.7,C79.1,C79.8,R18,I10,E16.1,D64.9,Z92.2,Z92.6,Z90.4,Z87.1,Z80.3,Z80.0,Z80.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.91,88.77,99.29,91.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88181173,3.0,64102,"0,885",2,3,NaN
2,116100,70825100,HOMBRE,1953-11-07,NINGUNO,CURICO,MOLINA,CHILE,FONASA INSTITUCIONAL - (MAI) B,DEL MAULE,"CENTRO ESPECIALIDADES (CDT, CRS, CONSULTORIO A...",PROGRAMADA,UROLOGÍA,HOSPITALIZACIÓN,2024-06-24,DESCONOCIDO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-07-03,DESCONOCIDO,DOMICILIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C79.0,C80.0,I10,E11.9,E66.0,D64.9,F10.8,Z90.5,Z86.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.23,88.75,88.01,87.41,89.52,89.65,90.59,

In [48]:
df.tail()

,COD_HOSPITAL,ID_BENEFICIARIO,SEXO,FECHA_NACIMIENTO,ETNIA,PROVINCIA,COMUNA,NACIONALIDAD,PREVISION,SERVICIO_SALUD,TIPO_PROCEDENCIA,TIPO_INGRESO,ESPECIALIDAD_MEDICA,TIPO_ACTIVIDAD,FECHA_INGRESO,SERVICIOINGRESO,FECHATRASLADO1,SERVICIOTRASLADO1,FECHATRASLADO2,SERVICIOTRASLADO2,FECHATRASLADO3,SERVICIOTRASLADO3,FECHATRASLADO4,SERVICIOTRASLADO4,FECHATRASLADO5,SERVICIOTRASLADO5,FECHATRASLADO6,SERVICIOTRASLADO6,FECHATRASLADO7,SERVICIOTRASLADO7,FECHATRASLADO8,SERVICIOTRASLADO8,FECHATRASLADO9,SERVICIOTRASLADO9,FECHAALTA,SERVICIOALTA,TIPOALTA,CONDICIONDEALTANEONATO1,PESORN1,SEXORN1,RN1ESTADO,CONDICIONDEALTANEONATO2,PESORN2,SEXORN2,RN2ESTADO,CONDICIONDEALTANEONATO3,PESORN3,SEXORN3,RN3ESTADO,CONDICIONDEALTANEONATO4,PESORN4,SEXORN4,RN4ESTADO,DIAGNOSTICO1,DIAGNOSTICO2,DIAGNOSTICO3,DIAGNOSTICO4,DIAGNOSTICO5,DIAGNOSTICO6,DIAGNOSTICO7,DIAGNOSTICO8,DIAGNOSTICO9,DIAGNOSTICO10,DIAGNOSTICO11,DIAGNOSTICO12,DIAGNOSTICO13,DIAGNOSTICO14,DIAGNOSTICO15,DIAGNOSTICO16,DIAGNOSTICO17,DIAGNOSTICO18,DIAGNOSTICO19,DIAGNOSTICO20,DIAGNOSTICO21,DIAGNOSTICO22,DIAGNOSTICO23,DIAGNOSTICO24,DIAGNOSTICO25,DIAGNOSTICO26,DIAGNOSTICO27,DIAGNOSTICO28,DIAGNOSTICO29,DIAGNOSTICO30,DIAGNOSTICO31,DIAGNOSTICO32,DIAGNOSTICO33,DIAGNOSTICO34,DIAGNOSTICO35,PROCEDIMIENTO1,PROCEDIMIENTO2,PROCEDIMIENTO3,PROCEDIMIENTO4,PROCEDIMIENTO5,PROCEDIMIENTO6,PROCEDIMIENTO7,PROCEDIMIENTO8,PROCEDIMIENTO9,PROCEDIMIENTO10,PROCEDIMIENTO11,PROCEDIMIENTO12,PROCEDIMIENTO13,PROCEDIMIENTO14,PROCEDIMIENTO15,PROCEDIMIENTO16,PROCEDIMIENTO17,PROCEDIMIENTO18,PROCEDIMIENTO19,PROCEDIMIENTO20,PROCEDIMIENTO21,PROCEDIMIENTO22,PROCEDIMIENTO23,PROCEDIMIENTO24,PROCEDIMIENTO25,PROCEDIMIENTO26,PROCEDIMIENTO27,PROCEDIMIENTO28,PROCEDIMIENTO29,PROCEDIMIENTO30,MEDICOINTERV1_ENCRIPTADO,FECHAPROCEDIMIENTO1,FECHAINTERV1,ESPECIALIDADINTERVENCION,MEDICOALTA_ENCRIPTADO,USOSPABELLON,IR_29301_COD_GRD,IR_29301_PESO,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD,HOSPPROCEDENCIA
3890,114101,79515476,HOMBRE,1966-08-11,NINGUNO,CORDILLERA,PUENTE ALTO,CHILE,FONASA INSTITUCIONAL - (MAI) D,METROPOLITANO SURORIENTE,SERVICIO EMERGENCIA (DOMICILIO),URGENCIA,MEDICINA INTERNA,HOSPITALIZACIÓN,2024-08-26,DERIVACIÓN MÉDICO QUIRÚRGICO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-27,DERIVACIÓN MÉDICO QUIRÚRGICO,HOSPITALIZACIÓN DOMICILIARIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C78.0,C64,C77.1,J96.09,I10,Z92.2,Z88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93.96,87.41,93.94,99.19,89.66,90.59,93.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82069545,NaN,44143,"1,3157",3,3,NaN
3891,114103,70462207,MUJER,1973-11-16,NINGUNO,SANTIAGO,LA PINTANA,CHILE,FONASA INSTITUCIONAL - (MAI) D,METROPOLITANO SURORIENTE,"CENTRO ESPECIALIDADES (CDT, CRS, CONSULTORIO A...",PROGRAMADA,CIRUGÍA GENERAL,HOSPITALIZACIÓN,2024-08-13,CIRUGÍA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-08-14,CIRUGÍA,DOMICILIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C78.2,C50.2,C77.3,Z90.1,Z90.7,Z92.4,Z92.3,Z92.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.20,87.44,34.04,97.41,99.21,99.29,93.11,93.59,99.19,89.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,94564042.0,NaN,DESCONOCIDA,CIRUGÍA GENERAL,94564042,1.0,44142,"0,9209",2,2,NaN
3892,116105,69757930,MUJER,1955-12-27,NINGUNO,TALCA,CUREPTO,CHILE,FONASA INSTITUCIONAL - (MAI) C,DEL MAULE,"CENTRO ESPECIALIDADES (CDT, CRS, CONSULTORIO A...",PROGRAMADA,CIRUGÍA DE TÓRAX,HOSPITALIZACIÓN,2024-01-01,CIRUGÍA,2024-01-02,UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO,2024-01-07,AGUDOS CIRUGÍA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-01-08,AGUDOS CIRUGÍA,DOMICILIO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C78.2,C65,C66,J90,J44.8,M19.80,Z92.6,M31.1,I10,N39.3,E11.6,E66.8,E03.8,NaN

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3895 entries, 0 to 3894
Columns: 129 entries, COD_HOSPITAL to HOSPPROCEDENCIA
dtypes: float64(50), int64(6), object(73)
memory usage: 3.8+ MB


In [51]:
#df.describe(include="all").T
df.describe()

,COD_HOSPITAL,ID_BENEFICIARIO,FECHATRASLADO8,SERVICIOTRASLADO8,FECHATRASLADO9,SERVICIOTRASLADO9,CONDICIONDEALTANEONATO1,PESORN1,RN1ESTADO,CONDICIONDEALTANEONATO2,PESORN2,SEXORN2,RN2ESTADO,CONDICIONDEALTANEONATO3,PESORN3,SEXORN3,RN3ESTADO,CONDICIONDEALTANEONATO4,PESORN4,SEXORN4,RN4ESTADO,DIAGNOSTICO35,PROCEDIMIENTO1,PROCEDIMIENTO5,PROCEDIMIENTO6,PROCEDIMIENTO7,PROCEDIMIENTO8,PROCEDIMIENTO9,PROCEDIMIENTO10,PROCEDIMIENTO11,PROCEDIMIENTO12,PROCEDIMIENTO13,PROCEDIMIENTO14,PROCEDIMIENTO15,PROCEDIMIENTO16,PROCEDIMIENTO17,PROCEDIMIENTO18,PROCEDIMIENTO19,PROCEDIMIENTO20,PROCEDIMIENTO21,PROCEDIMIENTO22,PROCEDIMIENTO23,PROCEDIMIENTO24,PROCEDIMIENTO25,PROCEDIMIENTO26,PROCEDIMIENTO27,PROCEDIMIENTO28,PROCEDIMIENTO29,PROCEDIMIENTO30,MEDICOINTERV1_ENCRIPTADO,FECHAPROCEDIMIENTO1,MEDICOALTA_ENCRIPTADO,USOSPABELLON,IR_29301_COD_GRD,IR_29301_SEVERIDAD,IR_29301_MORTALIDAD
count,3895.000000,3.895000e+03,0.0,0.0,0.0,0.0,0.0,2.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3894.000000,3203.000000,3002.000000,2784.000000,2562.000000,2329.000000,2093.000000,1893.000000,1694.000000,1503.000000,1304.000000,1129.000000,975.000000,865.000000,742.000000,659.000000,568.000000,478.000000,395.000000,330.00000,292.000000,248.000000,212.000000,187.000000,158.000000,141.000000,116.000000,2.069000e+03,0.0,3.895000e+03,2348.000000,3895.000000,3895.000000,3895.000000
mean,113785.363543,7.590430e+07,NaN,NaN,NaN,NaN,NaN,2685.000000,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.128765,87.360777,88.930043,89.827252,89.897760,90.038626,89.932585,89.901231,89.106104,89.528017,89.353405,89.500797,89.171118,88.776000,89.304596,89.490653,87.523627,89.297741,88.641873,89.33097,88.351849,87.764194,89.310236,88.266043,86.253038,89.248298,87.378276,8.131440e+07,NaN,8.429132e+07,1.382453,69256.288062,2.086264,2.382285
std,6937.763344,7.135108e+06,NaN,NaN,NaN,NaN,NaN,1011.162697,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.660762,17.774318,16.194224,15.044043,15.166035,14.721465,14.895018,15.042325,16.435811,15.501649,15.590389,14.608572,16.055464,16.141085,15.373607,14.815370,17.878070,14.650118,17.048537,16.15722,17.167946,17.517353,15.500002,18.714210,19.765875,15.095858,15.694969,8.861010e+06,NaN,9.399934e+06,0.860673,50866.701487,0.853257,0.827397
min,101100.000000,6.699417e+07,NaN,NaN,NaN,NaN,NaN,1970.000000,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.170000,0.090000,0.140000,0.140000,0.170000,0.140000,0.140000,0.140000,0.140000,0.140000,0.010000,0.140000,0.170000,0.090000,0.140000,0.140000,0.170000,0.170000,0.170000,0.17000,0.170000,0.170000,0.170000,0.170000,0.170000,0.170000,0.170000,6.708370e+07,NaN,6.700080e+07,1.000000,11012.000000,0.000000,0.000000
25%,110100.000000,7.052111e+07,NaN,NaN,NaN,NaN,NaN,2327.500000,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.440000,88.010000,88.430000,89.050000,89.130000,89.520000,89.520000,89.520000,89.520000,89.520000,89.390000,89.390000,89.520000,89.520000,89.540000,89.540000,89.520000,89.520000,89.520000,89.61000,89.520000,89.450000,89.535000,89.540000,89.390000,89.620000,89.067500,7.350094e+07,NaN,7.715932e+07,1.000000,43105.000000,2.000000,2.000000
50%,114101.000000,7.441119e+07,NaN,NaN,NaN,NaN,NaN,2685.000000,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54.300000,90.590000,91.320000,93.110000,93.110000,93.110000,93.110000,93.010000,93.110000,91.390000,91.390000,91.330000,91.390000,91.330000,91.390000,91.390000,91.320000,91.320000,91.330000,91.34500,91.320000,93.010000,92.595000,92.140000,90.590000,93.110000,90.475000,8.106750e+07,NaN,8.372211e+07,1.000000,64102.000000,2.000000,3.000000
75%,118100.000000,7.928883e+07,NaN,NaN,NaN,NaN,NaN,3042.500000,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,87.410000,99.170000,99.190000,99.190000,99.190000,99.180000,99.180000,99.150000,99.090000,99.040000,97.640000,96.590000,96.590000,96.590000,96.590000,96.420000,94.107500,96.587500,96.590000,96.59000,96.070000,96.150000,96.140000,93.960000,93.875000,94.3

In [52]:
#FECHATRASLADO8

categorias = df.select_dtypes(exclude="number").apply(lambda x: x.unique())

for col, vals in categorias.items():
    print(f"Valores únicos en '{col}': {vals}")

Valores únicos en 'SEXO': ['MUJER' 'HOMBRE' 'DESCONOCIDO']
Valores únicos en 'FECHA_NACIMIENTO': ['1952-11-11' '1948-02-02' '1953-11-07' ... '1973-11-16' '1955-12-27'
 '1993-09-21']
Valores únicos en 'ETNIA': ['NINGUNO' 'MAPUCHE' 'AYMARA' 'DIAGUITA' 'RAPA NUI (PASCUENSE)' 'COLLA'
 'YAGÁN (YÁMANA)']
Valores únicos en 'PROVINCIA': ['CORDILLERA' 'SANTIAGO' 'CURICO' 'OSORNO' 'LIMARI' 'LLANQUIHUE'
 'VALDIVIA' 'VALPARAISO' 'CAUTIN' 'ITATA' 'MAIPO' 'CONCEPCION' 'QUILLOTA'
 'COIHAIQUE' 'PALENA' 'IQUIQUE' 'TALCA' 'BIO-BIO' 'ELQUI' 'TALAGANTE'
 'DIGUILLÍN' 'COPIAPO' 'LINARES' 'RANCO' 'COLCHAGUA' 'PUNILLA'
 'MARGA MARGA' 'HUASCO' 'SAN ANTONIO' 'ANTOFAGASTA' 'CHAÑARAL'
 'MAGALLANES' 'ARAUCO' 'ARICA' 'SAN FELIPE' 'CACHAPOAL' 'MELIPILLA'
 'CHACABUCO' 'MALLECO' 'LOS ANDES' 'PETORCA' 'CHILOE' 'CAUQUENES'
 'CARDENAL CARO' 'DESCONOCIDO' 'ISLA DE PASCUA' 'GENERAL CARRERA' 'CHOAPA'
 'EL LOA' 'AISEN' 'TOCOPILLA' 'TAMARUGAL' 'ULTIMA ESPERANZA'
 'CAPITAN PRAT']
Valores únicos en 'COMUNA': ['PUENTE ALTO' 'LA 

In [53]:
numeric_cols = df.select_dtypes(include=np.number)

z_scores = (numeric_cols - numeric_cols.mean()) / numeric_cols.std()

outliers_por_col = (z_scores.abs() > 3).sum()

outliers_pct = ((z_scores.abs() > 3).sum() / len(df)) * 100

outliers_summary = pd.DataFrame({
    "cantidad_outliers": outliers_por_col,
    "porcentaje_outliers": outliers_pct.round(2).astype(str) + "%"
})

print(outliers_summary)

                          cantidad_outliers porcentaje_outliers
COD_HOSPITAL                              5               0.13%
ID_BENEFICIARIO                         102               2.62%
FECHATRASLADO8                            0                0.0%
SERVICIOTRASLADO8                         0                0.0%
FECHATRASLADO9                            0                0.0%
SERVICIOTRASLADO9                         0                0.0%
CONDICIONDEALTANEONATO1                   0                0.0%
PESORN1                                   0                0.0%
RN1ESTADO                                 0                0.0%
CONDICIONDEALTANEONATO2                   0                0.0%
PESORN2                                   0                0.0%
SEXORN2                                   0                0.0%
RN2ESTADO                                 0                0.0%
CONDICIONDEALTANEONATO3                   0                0.0%
PESORN3                                 

In [54]:
def iqr(s):
    q1, q3 = s.quantile([.25, .75])
    return q3 - q1

print(df.select_dtypes(include=np.number).apply(iqr))

COD_HOSPITAL                8.000000e+03
ID_BENEFICIARIO             8.767722e+06
FECHATRASLADO8                       NaN
SERVICIOTRASLADO8                    NaN
FECHATRASLADO9                       NaN
SERVICIOTRASLADO9                    NaN
CONDICIONDEALTANEONATO1              NaN
PESORN1                     7.150000e+02
RN1ESTADO                   0.000000e+00
CONDICIONDEALTANEONATO2              NaN
PESORN2                              NaN
SEXORN2                              NaN
RN2ESTADO                            NaN
CONDICIONDEALTANEONATO3              NaN
PESORN3                              NaN
SEXORN3                              NaN
RN3ESTADO                            NaN
CONDICIONDEALTANEONATO4              NaN
PESORN4                              NaN
SEXORN4                              NaN
RN4ESTADO                            NaN
DIAGNOSTICO35                        NaN
PROCEDIMIENTO1              5.197000e+01
PROCEDIMIENTO5              1.116000e+01
PROCEDIMIENTO6  

In [55]:
def safe_skew(s):
    try:
        return s.skew()
    except Exception:
        return np.nan
    
print(df.select_dtypes(include=np.number).apply(safe_skew))

COD_HOSPITAL                2.593994
ID_BENEFICIARIO             1.318636
FECHATRASLADO8                   NaN
SERVICIOTRASLADO8                NaN
FECHATRASLADO9                   NaN
SERVICIOTRASLADO9                NaN
CONDICIONDEALTANEONATO1          NaN
PESORN1                          NaN
RN1ESTADO                        NaN
CONDICIONDEALTANEONATO2          NaN
PESORN2                          NaN
SEXORN2                          NaN
RN2ESTADO                        NaN
CONDICIONDEALTANEONATO3          NaN
PESORN3                          NaN
SEXORN3                          NaN
RN3ESTADO                        NaN
CONDICIONDEALTANEONATO4          NaN
PESORN4                          NaN
SEXORN4                          NaN
RN4ESTADO                        NaN
DIAGNOSTICO35                    NaN
PROCEDIMIENTO1             -0.210134
PROCEDIMIENTO5             -2.771185
PROCEDIMIENTO6             -3.152982
PROCEDIMIENTO7             -3.476705
PROCEDIMIENTO8             -3.561337
P

In [57]:
def frecuencias_categoricas(df, top_k=None):
    resultados = {}
    
    cat_cols = df.select_dtypes(exclude="number").columns
    for col in cat_cols:
        vc = df[col].value_counts(dropna=False)  # incluye NaN
        total = len(df)
        tabla = pd.DataFrame({
            "valor": vc.index.astype(str),
            "frecuencia": vc.values,
            "porcentaje": (vc.values / total * 100).round(2)
        })
        if top_k:
            tabla = tabla.head(top_k)
        resultados[col] = tabla.reset_index(drop=True)
    return resultados

cat_cols = df.select_dtypes(exclude="number").columns

frecuencias_categoricas(df.drop(columns=cat_cols), top_k=5)

{}